# Nuclio - Generator function

In [1]:
# nuclio: ignore
import nuclio

### Setups

In [2]:
%%nuclio cmd -c

# Utils
pip install pyarrow
pip install pyyaml --upgrade
pip install pandas
pip install pytimeparse

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install -i https://test.pypi.org/simple/ v3io-generator
pip install faker

## Function

In [37]:
import os
import datetime
import time
import yaml
import pandas as pd
import itertools

# MLRun
from mlrun import get_or_create_ctx

# DB Connection
import v3io_frames as v3f

# Data generator
from v3io_generator import metrics_generator, deployment_generator

### Helper functions

In [38]:
def _create_deployment():
    context.logger.info('Creating deployment')
    # Create meta-data factory
    dep_gen = deployment_generator.deployment_generator()
    faker=dep_gen.get_faker()

    # Design meta-data
    dep_gen.add_level(name='company',number=2,level_type=faker.company)
    dep_gen.add_level('data_center',number=2,level_type=faker.street_name)
    dep_gen.add_level('device',number=2,level_type=faker.msisdn)

    # Create meta-data
    deployment_df = dep_gen.generate_deployment()
    return deployment_df

In [39]:
def _is_deployment_exist(path):
    # Checking shared path for the devices table
    return os.path.exists(f'/v3io/bigdata/{path}')

In [40]:
def _get_deployment_from_kv(client, path):
    context.logger.info(f'Retrieving deployment from {path}')
    # Read the devices table from our KV store
    deployment_df = client.read(backend='kv', table=path)
    
    # Reset index to column
    deployment_df.index.name = 'device'
    deployment_df = deployment_df.reset_index()
    return deployment_df

In [41]:
def _save_deployment_to_kv(path, df, client=None):
    context.logger.info(f'Saving deployment to {path}')
    # Save deployment to our KV store
    client.write(backend='kv', table=path ,dfs=df, index_cols=['device'])

In [42]:
def get_or_create_deployment(path, client=None):
    deployment_df = None
    if client and _is_deployment_exist(path):
        # Get deployment from KV
        deployment_df = _get_deployment_from_kv(client, path)
    else:
        # Create deployment
        deployment_df = _create_deployment()
        
        context.logger.info(deployment_df)
        if client:
            _save_deployment_to_kv(path, deployment_df, client)

    return deployment_df

In [451]:
def set_indexes(df):
    df = df.set_index(['time', 'company', 'data_center', 'device'])
    return df

In [452]:
def save_metrics_to_tsdb(client, metrics: pd.DataFrame, metrics_table):
    client.write('tsdb', metrics_table, metrics)

In [546]:
def save_metrics_to_parquet(metrics, metrics_table):
#     df = pd.concat(itertools.chain(metrics))
    
    # Need to fix timestamps from ns to ms if we write to parquet
    df = metrics.reset_index()
    df['time'] = df.loc[:, 'time'].astype('datetime64[ms]')
    
    # Fix indexes
    df = set_indexes(df)
    
    # Save parquet
    first_timestamp = df.index[0][0].strftime('%Y%m%dT%H%M%S')
    last_timestamp = df.index[-1][0].strftime('%Y%m%dT%H%M%S')
    filename = first_timestamp + '-' + last_timestamp + '.parquet'
    filedir = os.path.join(os.getcwd(), metrics_table)
    filepath = os.path.join(filedir, filename)
    os.makedirs(filedir, exist_ok=True)
    with open(filepath, 'wb+') as f:
        df.to_parquet(f)
    
    return filepath

### Handler

In [547]:
v3c = v3f.Client(address='framesd:8081', container='bigdata')

def generator(context,
              metrics_configuration_file,
              initial_timestamp,
              use_tsdb,
              metrics_table,
              deployment_table,
              secs_to_generate = 180000):
    
    # load MLRUN runtime context (will be set by the runtime framework e.g. KubeFlow)
    context.logger.info('Loading parameters')
    metrics_configuration = yaml.safe_load(metrics_configuration_file.get())

    # Set V3IO Connection if needed
    client = None
    if use_tsdb:
        client = v3f.Client(address='framesd:8081', container='bigdata')
        client.create('tsdb', metrics_table, attrs={'rate': '1/s'}, if_exists=True)
    
    # Generate or create deployment
    deployment_df = get_or_create_deployment(deployment_table, client)
    context.logger.info(f'Deployment:\n{deployment_df}')

#     ## Set base initial values 
    deployment_df['cpu_utilization'] = 70
    deployment_df['latency'] = 0
    deployment_df['packet_loss'] = 0
    deployment_df['throughput'] = 290

    context.logger.info(f'Metrics Configuration:\n{metrics_configuration}')
    
#     # Create metrics generator
    start_time = datetime.datetime.fromtimestamp(initial_timestamp)
    end_time = (datetime.datetime.fromtimestamp(initial_timestamp)+datetime.timedelta(seconds=secs_to_generate))
    context.logger.info(f'Generating data from {start_time} to {end_time}')
    
    met_gen = metrics_generator.Generator_df(metrics_configuration, 
                                             user_hierarchy=deployment_df, 
                                             initial_timestamp=start_time)
    
    context.logger.info('Generator created')
    # Create metrics generator based on YAML configuration and deployment
    metrics = met_gen.generate_range(start_time=start_time,
                                     end_time=end_time,
                                     as_df=True,
                                     as_iterator=True)
    
    metrics = pd.concat(itertools.chain(metrics))
    metrics = metrics.reset_index()
    metrics = metrics.rename(columns={'timestamp': 'time'})
    metrics = metrics.set_index(['time', 'company', 'data_center', 'device'])
    context.logger.info(metrics.index.names)
    context.logger.info(f'Generated metrics:\nSample: {metrics.head(1)}')
    
    # Save Generated metrics
    if use_tsdb:
#         save_metrics_to_tsdb(client, metrics, metrics_table)
        client.write('tsdb', metrics_table, metrics)
        context.logger.info(f'Saved data to TSDB: {metrics_table}')
    else:
        data = save_metrics_to_parquet(metrics, metrics_table)
        context.log_artifact('metrics', src_path=data, target_path=os.path.join(metrics_table, os.path.basename(data)), upload=True)
        context.logger.info(f'Save data to Parquet: {data}')

In [548]:
# nuclio: end-code

In [549]:
# nuclio: ignore
from mlrun import new_function, code_to_function, mount_v3io, NewTask

In [550]:
# nuclio: ignore
params = {
    'use_tsdb': False,
    'deployment_table': 'netops_deployment',
    'metrics_table': 'netops_metrics',
    'initial_timestamp': (datetime.datetime.now()-datetime.timedelta(days=1)).timestamp(),
    'secs_to_generate': 10
}
inputs = {
    'metrics_configuration_file': os.path.join(os.getcwd(), 
                                              'configurations', 
                                              'metrics_configuration.yaml')
}

fn = new_function(runtime='', interactive=True)
task = NewTask(handler=generator, 
               params=params, 
               inputs=inputs,
               out_path='/User/mlrun-db/data/')
fn.run(task)

[mlrun] 2019-11-21 13:58:01,265 starting run generator uid=e771b91664ca464995d5be26be6e576a  -> 
Python> 2019-11-21 13:58:01,361 [info] Creating deployment
Python> 2019-11-21 13:58:03,494 [info]                    company      data_center         device
0  Mcdonald_Ltd             Scott_Fall       4500990616269
1  Mcdonald_Ltd             Scott_Fall       3083226030475
2  Mcdonald_Ltd             Crystal_Landing  2534551030081
3  Mcdonald_Ltd             Crystal_Landing  5041984634187
4  Morgan__Bush_and_Mcneil  Mason_Burg       9175859025474
5  Morgan__Bush_and_Mcneil  Mason_Burg       9083478070791
6  Morgan__Bush_and_Mcneil  Hansen_Stream    4344129471960
7  Morgan__Bush_and_Mcneil  Hansen_Stream    5311561585564
[mlrun] 2019-11-21 13:58:01,347 Loading parameters
[mlrun] 2019-11-21 13:58:03,507 Deployment:
                   company      data_center         device
0  Mcdonald_Ltd             Scott_Fall       4500990616269
1  Mcdonald_Ltd             Scott_Fall       3083226030475
2 

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...6e576a,0,Nov 21 13:58:01,completed,generator,repo=http://github.com/mlrun/demoscommit=ce048c7686aa5348e905d248598fede8418ed896kind=handlerowner=adminhost=jupyter-h4pye88pz3-itkjk-6bbcdb955c-j8xtr,metrics_configuration_file,use_tsdb=Falsedeployment_table=netops_deploymentmetrics_table=netops_metricsinitial_timestamp=1574258281.264221secs_to_generate=10,,metrics


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid e771b91664ca464995d5be26be6e576a 
[mlrun] 2019-11-21 13:58:03,772 run executed, status=completed
